In [ ]:
%pip install datasets
%pip install evaluate
%pip install transformers
%pip install torch
%pip install numpy
%pip install peft
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from datasets import load_dataset, DatasetDict, Dataset
from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoModelForCausalLM,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)
from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np
from transformers import pipeline
import pandas as pd
import json
import requests
from sklearn.model_selection import train_test_split

2024-11-19 14:43:27.160590: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <2D89372E-C77C-3351-8455-B2E540126864> /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torchvision/image.so
  Expected in:     <A84DFEFF-287E-3B94-A7DB-731FA5F9CBBC> /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/lib/libtorch_cpu.dylib'If you don't 

In [4]:
# !huggingface-cli login

In [ ]:
# Use a pipeline as a high-level helper
pipe = pipeline("text-generation", model="meta-llama/Llama-3.2-1B")

In [6]:
# # Step 1: Mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive', force_remount = True)

In [ ]:
url = "https://raw.githubusercontent.com/microsoft/NoFunEval/refs/heads/main/datasets/resource_util.jsonl"
data = [json.loads(line) for line in requests.get(url).text.split('\n')[:-1]]

# Convert to a pandas DataFrame
df = pd.DataFrame(data)

# Display the first few rows
print(df.head())


  non_functional_requirement  \
0                     memory   
1                     memory   
2                     memory   
3                     memory   
4                     memory   

                                              commit  \
0  https://github.com/aaronjwood/PortAuthority/co...   
1  https://github.com/aaronjwood/PortAuthority/co...   
2  https://github.com/aaronjwood/PortAuthority/co...   
3  https://github.com/aaronjwood/PortAuthority/co...   
4  https://github.com/vaginessa/TestBrowser-Light...   

                                  commit_message  \
0       '\\"Avoid potential resource leak\\n\\"'   
1                   '\\"Fix resource leak\\n\\"'   
2                     '\\"Fix memory leak\\n\\"'   
3  '\\"Fix resource leak in certain cases\\n\\"'   
4                 '\\"Fixed a memory leak\\n\\"'   

                                         source_code  \
0  package com.aaronjwood.portauthority.network;\...   
1  package com.aaronjwood.portauthority.runna

In [8]:
# Assuming df is already loaded as a pandas DataFrame
# Combine relevant columns (adjust as needed, here I'm using 'base_prompt' as input and 'target_code' as the output)
df['input_text'] = df['base_prompt'] + ' ' + df['coding_concepts'] + ' ' + df['chain_of_thought'] + ' ' + df['source_code']
df['output_text'] = df['target_code']

In [9]:
print(df.head())

  non_functional_requirement  \
0                     memory   
1                     memory   
2                     memory   
3                     memory   
4                     memory   

                                              commit  \
0  https://github.com/aaronjwood/PortAuthority/co...   
1  https://github.com/aaronjwood/PortAuthority/co...   
2  https://github.com/aaronjwood/PortAuthority/co...   
3  https://github.com/aaronjwood/PortAuthority/co...   
4  https://github.com/vaginessa/TestBrowser-Light...   

                                  commit_message  \
0       '\\"Avoid potential resource leak\\n\\"'   
1                   '\\"Fix resource leak\\n\\"'   
2                     '\\"Fix memory leak\\n\\"'   
3  '\\"Fix resource leak in certain cases\\n\\"'   
4                 '\\"Fixed a memory leak\\n\\"'   

                                         source_code  \
0  package com.aaronjwood.portauthority.network;\...   
1  package com.aaronjwood.portauthority.runna

In [ ]:
# Split the data
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['non_functional_requirement'])

In [11]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

In [14]:
print(train_dataset.column_names)

['non_functional_requirement', 'commit', 'commit_message', 'source_code', 'target_code', 'pl', 'chain_of_thought', 'one_shot', 'base_prompt', 'coding_concepts', 'classification_left_prompt', 'classification_left_label', 'classification_right_prompt', 'classification_right_label', 'input_text', 'output_text', '__index_level_0__']


In [15]:
# !huggingface-cli login
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B")

In [17]:
# Set the pad token to be the eos token (common approach for models without a pad token)
# tokenizer.pad_token = tokenizer.eos_token

# Now, you can proceed with tokenization
def tokenize_function(examples):
    return tokenizer(examples['input_text'], truncation=True, padding='max_length', max_length=2048)

# tokenized_train_data = train_data.map(tokenize_function, batched=True)
tokenized_train_data = train_dataset.map(tokenize_function, batched=True)

# Format the dataset for causal language modeling
tokenized_train_data = tokenized_train_data.map(
    lambda examples: {'labels': examples['input_ids']},
    batched=True
)

# Check tokenized data
print(tokenized_train_data[0])

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

{'non_functional_requirement': 'memory', 'commit': 'https://github.com/erikusaj/fdroidTvClient/commit/4a7f0ef9a213e43ab15843cf2473077a69bab37c', 'commit_message': '\'\\\\"Resolve memory issue when updating repo.\\\\n\\\\nSolves issue # 453. Previously', 'source_code': '/*\n * Copyright (C) 2010-12  Ciaran Gultnieks, ciaran@ciarang.com\n *\n * This program is free software; you can redistribute it and/or\n * modify it under the terms of the GNU General Public License\n * as published by the Free Software Foundation; either version 3\n * of the License, or (at your option) any later version.\n *\n * This program is distributed in the hope that it will be useful,\n * but WITHOUT ANY WARRANTY; without even the implied warranty of\n * MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the\n * GNU General Public License for more details.\n *\n * You should have received a copy of the GNU General Public License\n * along with this program; if not, write to the Free Software\n * Foundat

In [18]:
# Define the LoRA configuration
lora_config = LoraConfig(
    r=6,  # rank for the LoRA layers
    lora_alpha=16,  # scaling factor for LoRA
    lora_dropout=0.1,  # dropout rate for LoRA
    task_type="CAUSAL_LM"  # causal language modeling task
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./lora_model",
    evaluation_strategy="epoch",  # Evaluate every epoch
    learning_rate=2e-5,           # Learning rate
    per_device_train_batch_size=4,  # Batch size
    num_train_epochs=3,           # Number of epochs
    logging_dir='./logs',         # Where to save logs
    save_steps=500,               # Save checkpoints
    logging_steps=100,            # Log every 100 steps
    # fp16=True                     # Enable mixed precision (if supported)
)

# Set up the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_data,
    tokenizer=tokenizer
)

In [21]:
# Train the model
trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/30 [00:00<?, ?it/s]

: 